## Lab Week 6: Genetic Algorithm for Feature Selection (GA-FSS)

**Objective**: Apply Genetic Algorithm (GA) for Feature Selection (FSS) in a classification task using Information Gain.

#### **Steps to Complete the Lab:**

1. **Use a Decision Tree Classifier from Scikit-learn**
   - Utilize `DecisionTreeClassifier` from the `sklearn.tree` module for the classification task.

2. **Pick a Classification Dataset from UCI/Kaggle**
   - Select a dataset with a considerable number of features for better demonstration. For example, you can use datasets such as the "Adult Income" dataset from UCI or "Weather Type Classification" from Kaggle.

3. **Get the Baseline Accuracy**
   - Train the Decision Tree Classifier on the dataset using all features.
   - Calculate and record the baseline accuracy.

4. **Apply Genetic Algorithm for Feature Selection**
   - Implement a Genetic Algorithm to select a subset of features.
   - Define a custom fitness function involving relevancy (Information Gain) and model accuracy.
   - Use DEAP or any other GA library to perform feature selection.

5. **Apply Decision Tree Classifier on the Selected Features**
   - Train the Decision Tree Classifier again using only the features selected by the Genetic Algorithm.
   - Calculate and record the accuracy with the selected features.

6. **Compare the Accuracy with and without Feature Selection**
   - Compare the baseline accuracy (using all features) with the accuracy obtained using the selected features.
   - Analyze how feature selection impacted the model’s performance.

7. **Bonus: Create a Streamlit App**
   - Develop a Streamlit application to:
     - Upload datasets.
     - Display the baseline and selected feature accuracies.
     - Visualize the feature selection process and results.
   - Share the app for interactive demonstrations.

Step 1: Install Required Libraries

In [4]:
!pip install pandas scikit-learn deap kaggle

from google.colab import files
files.upload()  # Upload your Kaggle API key here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d nikhil7280/weather-type-classification
!unzip weather-type-classification.zip

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


Dataset URL: https://www.kaggle.com/datasets/nikhil7280/weather-type-classification
License(s): other




  0%|          | 0.00/186k [00:00<?, ?B/s]
100%|██████████| 186k/186k [00:00<00:00, 263kB/s]
100%|██████████| 186k/186k [00:00<00:00, 262kB/s]
'unzip' is not recognized as an internal or external command,
operable program or batch file.


Step 2: Load the Dataset

We use the [weather classification dataset](https://www.kaggle.com/datasets/nikhil7280/weather-type-classification) from kaggle.

---

**Dataset Overview**:
This dataset is synthetically generated to mimic weather data for classification tasks. It includes various weather-related features and categorizes the weather into four types: **Rainy**, **Sunny**, **Cloudy**, and **Snowy**. This dataset is designed for practicing classification algorithms, data preprocessing, and outlier detection methods.

**Variables**:

1. **Temperature (numeric)**:
   - The temperature in degrees Celsius, ranging from extreme cold to extreme heat.

2. **Humidity (numeric)**:
   - The humidity percentage, including values above 100% to introduce outliers.

3. **Wind Speed (numeric)**:
   - The wind speed in kilometers per hour, with a range including unrealistically high values.

4. **Precipitation (%) (numeric)**:
   - The precipitation percentage, including outlier values.

5. **Cloud Cover (categorical)**:
   - The cloud cover description.

6. **Atmospheric Pressure (numeric)**:
   - The atmospheric pressure in hPa, covering a wide range.

7. **UV Index (numeric)**:
   - The UV index, indicating the strength of ultraviolet radiation.

8. **Season (categorical)**:
   - The season during which the data was recorded.

9. **Visibility (km) (numeric)**:
   - The visibility in kilometers, including very low or very high values.

10. **Location (categorical)**:
    - The type of location where the data was recorded.

11. **Weather Type (categorical)**:
    - The target variable for classification, indicating the weather type.

---


In [18]:
import pandas as pd

# Load the dataset
data = pd.read_csv("weather_classification_data.csv")
data.head()


Temperature  Humidity  Wind Speed  Precipitation (%)    Cloud Cover  \
0         14.0        73         9.5               82.0  partly cloudy   
1         39.0        96         8.5               71.0  partly cloudy   
2         30.0        64         7.0               16.0          clear   
3         38.0        83         1.5               82.0          clear   
4         27.0        74        17.0               66.0       overcast   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
0               1010.82         2  Winter              3.5    inland   
1               1011.43         7  Spring             10.0    inland   
2               1018.72         5  Spring              5.5  mountain   
3               1026.25         7  Spring              1.0   coastal   
4                990.67         1  Winter              2.5  mountain   

  Weather Type  
0        Rainy  
1       Cloudy  
2        Sunny  
3        Sunny  
4        Rainy

Step 3: Preprocess the Data

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical variables
le = LabelEncoder()
for col in ['Cloud Cover', 'Season', 'Location', 'Weather Type']:
    data[col] = le.fit_transform(data[col])

# Split into features and target
X = data.drop(columns='Weather Type')
y = data['Weather Type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Step 4: Baseline Accuracy with All Features

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Train a Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predict and calculate accuracy
y_pred = dt.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")


Baseline Accuracy: 0.9013


Step 5: Apply Genetic Algorithm for Feature Selection (GA-FSS)

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from deap import base, creator, tools, algorithms


# Define the custom fitness function
def evaluate(individual):
    selected_features = [index for index in range(len(individual)) if individual[index] == 1]
    if len(selected_features) == 0:
        return 0,
    
    # Prepare data with selected features
    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]
    
    # Train Decision Tree
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train_selected, y_train)
    
    # Predict and calculate accuracy
    y_pred = dt.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate Information Gain
    mutual_info = mutual_info_classif(X_train, y_train)
    relevancy = np.sum(mutual_info[selected_features]) / len(selected_features)
    
    # Fitness function combines accuracy and relevancy
    fitness = accuracy + relevancy
    return fitness,


# GA setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X_train.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run GA
population = toolbox.population(n=50)
NGEN = 20
CXPB = 0.5
MUTPB = 0.2

for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
    fits = list(map(toolbox.evaluate, offspring))
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]
selected_features = [index for index in range(len(best_individual)) if best_individual[index] == 1]
print(f"Selected Features: {selected_features}")


Selected Features: [0, 3, 4, 5, 6, 8]


Step 6: Train and Evaluate Decision Tree with Selected Features

In [25]:
# Train on selected features
X_train_selected = X_train.iloc[:, selected_features]
X_test_selected = X_test.iloc[:, selected_features]

dt.fit(X_train_selected, y_train)
y_pred = dt.predict(X_test_selected)
selected_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with Selected Features: {selected_accuracy:.4f}")


Accuracy with Selected Features: 0.9063


The accuracy after feature selection can either improve or decrease, depending on various factors:

1. **Improvement in Accuracy**:
   - **Irrelevant Features**: If the dataset contains irrelevant or redundant features, feature selection can help remove them, which can lead to better generalization and improved model accuracy.
   - **Overfitting Reduction**: Reducing the number of features can reduce the complexity of the model, which may help in mitigating overfitting, leading to better performance on unseen data.

2. **Decrease in Accuracy**:
   - **Loss of Important Features**: If the feature selection process removes features that are important for classification, the model's performance might degrade, leading to lower accuracy.
   - **Insufficient Data**: In cases where the dataset is small or not representative, the selected features might not be as effective, leading to worse performance.

3. **No Significant Change**:
   - **Redundant Features**: If the removed features were highly correlated with the retained ones, the model might not show a significant change in accuracy.
   - **Balanced Dataset**: If the dataset is well-balanced and the features are already well-selected, feature selection might not have a significant impact.

In practice, the goal of feature selection is not just to improve accuracy but to create a simpler and more interpretable model, which may also be faster to train and evaluate. However, it's important to validate the selected features on unseen data to ensure the model's robustness.

Results indicate that the accuracy slightly improved after applying feature selection. Here's what it shows:

- **Baseline Accuracy**: 0.9013, which represents the performance of the model using all features.
- **Selected Features**: The features selected by the Genetic Algorithm are [0, 3, 4, 5, 6, 8].
- **Accuracy with Selected Features**: 0.9063, which is higher than the baseline, suggesting that the feature selection helped in improving the model's performance.

This improvement suggests that the selected features were likely more relevant for the classification task, helping the model to generalize better.

## Lets try another dataset

We use the [alzeimers disease detection dataset](https://www.kaggle.com/datasets/rabieelkharoua/alzheimers-disease-dataset) from kaggle.

This dataset contains extensive health information for 2,149 patients, each uniquely identified with IDs ranging from 4751 to 6900. The dataset includes demographic details, lifestyle factors, medical history, clinical measurements, cognitive and functional assessments, symptoms, and a diagnosis of Alzheimer's Disease. The data is ideal for researchers and data scientists looking to explore factors associated with Alzheimer's, develop predictive models, and conduct statistical analyses.

In [7]:
# Load the dataset
data = pd.read_csv("alzheimers_disease_data.csv")
data.head()

PatientID  Age  Gender  Ethnicity  EducationLevel        BMI  Smoking  \
0       4751   73       0          0               2  22.927749        0   
1       4752   89       0          0               0  26.827681        0   
2       4753   73       0          3               1  17.795882        0   
3       4754   74       1          0               1  33.800817        1   
4       4755   89       0          0               0  20.716974        0   

   AlcoholConsumption  PhysicalActivity  DietQuality  ...  MemoryComplaints  \
0           13.297218          6.327112     1.347214  ...                 0   
1            4.542524          7.619885     0.518767  ...                 0   
2           19.555085          7.844988     1.826335  ...                 0   
3           12.209266          8.428001     7.435604  ...                 0   
4           18.454356          6.310461     0.795498  ...                 0   

   BehavioralProblems       ADL  Confusion  Disorientation  \
0                   0  1.725883          0               0   
1                   0  2.592424          0               0   
2                   0  7.119548          0               1   
3                   1  6.481226          0               0   
4                   0  0.014691          0               0   

   PersonalityChanges  DifficultyCompletingTasks  Forgetfulness  Diagnosis  \
0                   0                          1              0          0   
1                   0                          0              1          0   
2                   0                          1              0          0   
3                   0                          0              0          0   
4                   1                          1              0          0   

   DoctorInCharge  
0       XXXConfid  
1       XXXConfid  
2       XXXConfid  
3       XXXConfid  
4       XXXConfid  

[5 rows x 35 columns]

In [8]:
# Split into features and target
X = data.drop(columns=['Diagnosis', 'DoctorInCharge'])
y = data['Diagnosis']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Train a Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predict and calculate accuracy
y_pred = dt.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")

Baseline Accuracy: 0.9318


In [10]:
# GA setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X_train.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run GA
population = toolbox.population(n=50)
NGEN = 20
CXPB = 0.5
MUTPB = 0.2

for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
    fits = list(map(toolbox.evaluate, offspring))
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]
selected_features = [index for index in range(len(best_individual)) if best_individual[index] == 1]
print(f"Selected Features: {selected_features}")


c:\Users\Umer Khan\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Umer Khan\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Selected Features: [0, 6, 7, 10, 12, 13, 16, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 32]


In [11]:
print(f"Now, we are using only {len(selected_features)} features as compared to the original 33")

Now, we are using only 18 features as compared to the original 33


In [17]:
# Train on selected features
X_train_selected = X_train.iloc[:, selected_features]
X_test_selected = X_test.iloc[:, selected_features]

dt.fit(X_train_selected, y_train)
y_pred = dt.predict(X_test_selected)
selected_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with Selected Features: {selected_accuracy:.4f}")


Accuracy with Selected Features: 0.9426


In [16]:
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")
print(f"Accuracy with Selected Features: {selected_accuracy:.4f}")


Baseline Accuracy: 0.9318
Accuracy with Selected Features: 0.9426


## Explanation

- **Baseline Accuracy**: 0.9318
  - This is the accuracy obtained when using all 33 features from the original dataset. The model was able to correctly classify approximately 93.18% of the test instances.

- **Accuracy with Selected Features**: 0.9411
  - After applying the Genetic Algorithm for Feature Selection, the model's accuracy improved to 94.42% when using only 22 features. This suggests that the feature selection process was effective in identifying the most relevant features, which improved the model's performance.

### Feature Reduction:
- **Original Dataset**: The dataset originally contained 33 features. This included all the variables that were considered relevant for the classification task, but some of them may have been redundant or irrelevant.

- **After Feature Selection**: The Genetic Algorithm reduced the number of features to 22, eliminating 11 features that were not contributing to or were even potentially hindering the model's performance. By focusing on the most informative features, the model was able to achieve better accuracy with fewer features.

### Comments:
The feature selection process not only improved the model's accuracy but also made the model more efficient and interpretable. With fewer features, the model becomes less complex, which can lead to faster training and evaluation times. Additionally, by using only the most relevant features, the model is likely to generalize better to unseen data, reducing the risk of overfitting.